In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, f1_score, roc_auc_score, mean_absolute_error
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

from matplotlib import pyplot as plt
from sklearn.utils import shuffle

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [ ]:
'''przykład 1 - klasyfikacja kwiatów irysa

    1. Za pomocą pakietu scikit-learn załadujemy dane ze zbioru IRIS.
    2. Przekonwertujemy je do postaci Data Frame, żeby móc wygodnie operować na danych.
    3. Przeprowadzimy podział na zbiór treningowy i testowy.
    4. Zamienimy wektor wartości oczekiwanych ("Y") na format one-hot encoding, żeby zrozumieć zasadę działania funkcji Softmax.
    5. Przprowadzimy normalizację danych.'''

In [12]:
iris = load_iris()
iris_X = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_Y = pd.Series(iris.target)

NameError: name 'load_iris' is not defined

In [ ]:
'''jak wygląda zbiór danych?'''

iris_X.head(3)

In [ ]:
iris_X.describe()

In [ ]:
iris_Y.head(3)

In [ ]:
iris_Y.value_counts(normalize=True)

In [ ]:
iris_Y.value_counts()

In [ ]:
iris.target_names

In [ ]:
# do celów edukacyjnych: zamieńmy zmienną zależną/ odpowiedzi "Y" na postać one-hot encoding, zobaczymy jak zachowuje się funkcja softmax

iris_Y_one_hot = tf.keras.utils.to_categorical(iris_Y)

In [ ]:
'''podział danych na uczące i treningowe'''

# zazwyczaj robi się to kilka razy, w ramach procedury "walidacji krzyżowej" (ang. cross validation). Dla celów edukacyjnych,
# w naszym przypadku dokonamy jednokrotnego podziału na zbiór uczący i testowy

iris_X_train, iris_X_test, iris_Y_train, iris_Y_test = train_test_split(iris_X, iris_Y_one_hot, train_size=0.8, random_state=123)

In [ ]:
iris_X_train.shape, iris_Y_train.shape

In [ ]:
'''normalizacja danych'''

# sieć neuronowa jest podatna na "wysycenie gradientów" (ang. gradient vanishing), tj. sytuację w której duże wartości
# wpływają na szybkie odiągnięcie maksimum przez funkcję aktywacji. 
# W tym celu normalizujemy dane z użyciem uczącego się algorytmu skalującego (ang. scaler)
# Uczymy "StandardScaler na danych treningoeych, a potem przenosimy tę wiedzę na dane testowe,
# żeby oddać rzeczywistą sytuację

scaler = StandardScaler()
iris_X_trains = scaler.fit_transform(iris_X_train)
iris_X_trains = pd.DataFrame(iris_X_trains, columns=iris_X_train.columns)

In [ ]:
iris_X_trains.describe()

In [ ]:
'''Pierwsza sieć neuronowa'''

# sieć z użyciem Tensorflow - będzie się składać z 2 warstw:
# 1. Warstwy ukrytej - przyjmującej cechy opisujące kwiaty irysa
# 2. Warstwy wyjściowej - dokonującej klasyfikacji na 3 klasy


In [ ]:
no = iris_Y_one_hot.shape[1] # liczba klas
nx = iris_X_train.shape[1] # liczba cech
nh = 8 # liczba neuronów w warstwie ukrytej

W1 = tf.Variable(np.random.randn(nx, nh), name='W1')
b1 = tf.Variable(np.random.randn(1, nh), name='b1')

W2 = tf.Variable(np.random.randn(nh, no), name='W2')
b2 = tf.Variable(np.random.randn(1, no), name='b2')

lossf = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(momentum = 0.9)


In [ ]:
epochs = 15
batches = 6
beta = 0.001
nbatch = iris_X_train.shape[0]//batches

In [ ]:
losses = []
accuracies = []
for e in range(epochs):
    iris_X_trains, iris_Y_train = shuffle(iris_X_trains, iris_Y_train)
    offset = 0 
    for b in range(batches):
        end = offset + nbatch
        X, Y = iris_X_trains.iloc[offset:end, :], iris_Y_train[offset:end, :]

        with tf.GradientTape() as tape:
            h1 = tf.matmul(X, W1) + b1
            z1 = tf.nn.relu(h1)

            h2 = tf.matmul(z1, W2) + b2
            yhat = tf.nn.softmax(h2)

           
            loss = lossf(Y, yhat)
            reg = beta * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(b2))
            total_loss = loss + reg

            losses.append(total_loss.numpy())

            acc = tf.keras.metrics.categorical_accuracy(Y, yhat)
            accuracies.append(np.mean(acc.numpy()))

        gradients = tape.gradient(loss, [W1, b1, W2, b2])
        optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2]))
        offset = end - 1
       

In [ ]:
plt.plot(losses)
plt.title("Funkcja kosztu w czasie dla prostej sieci")
plt.xlabel("Numer iteracji")
plt.ylabel("Wartość funkcji kosztu")

In [ ]:
plt.plot(accuracies)
plt.title("Dokładność w czasie dla prostej sieci")
plt.xlabel("Numer iteracji")
plt.ylabel("Dokładność")

In [ ]:
'''Wykorzystanie gotowej biblioteki'''

# oczywiście przedstawiony wyżej sposób tworzenia sieci ma wyłącznie charakter podglądowy

'''Kolejność działań jest następująca:
    1. Zainicjujmy obiekt klasy 'Sequential' - jest to pusty obiekt sieci neuronowej, w której kolejne warstwy są układane jedna po drugiej.
    2. Następnie za pomocą metody 'add' dodajemy kolejne warstwy ukryte i końcową, o ilości neuronów równej ilości klas do przewidzenia.
    3. W zwykłej sieci klasyfikacyjnej (Multi Layer Perceptron) podstawę architektury stanowi wartwa typu 'Dense').
    4. W pierwszej warstwie należy wyspecyfikować:
        - 'input_dim' czyli liczbę atrybutów/ cech obiektów (wierszy) danych
        - liczbę neuronów 'units' 
        - funkcję aktywacji 'activation'
    5. Po zdefiniowaniu sieci kompilujemy ją za pomocą metody 'compile', w której należy podać:
        - optimizer - algorytm optymalizacyjny, którym będzie się posługiwać sieć
        - loss - funkcję kosztu, którą sieć będzie minimalizować
        - metrics - metrykę, którą będziemy monitorować w czasie uczenia
        
    Zreplikujemy sieć, którą wcześniej napisaliśmy ręcznie, żeby porównać wyniki'''


In [ ]:
'''Zreplikujemy sieć, którą wcześniej napisaliśmy ręcznie, żeby porównać wyniki'''

nh, nx

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(input_dim=nx, units=nh, activation='relu'))
model.add(tf.keras.layers.Dense(units=no, activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, rankdir='LR')

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(iris_X_trains, iris_Y_train, epochs=15, batch_size=24)

In [ ]:
plt.plot(history.history["loss"])
plt.title("Funkcja kosztu, gdy używamy biblioteki")
plt.xlabel("Iteracja")
plt.ylabel("Koszt")

In [ ]:
plt.plot(history.history["accuracy"])
plt.title("Trafność, gdy używamy biblioteki")
plt.xlabel("Iteracja")
plt.ylabel("Trafność")

In [ ]:
'''Predykcja na zbiorze testowym z wykorzystaniem gotowej biblioteki'''

# teraz dokonamy predykcjina zbiorze testowym, żeby ocenić trafność klasyfikacji naszego modelu.
# ważne jest, aby dokonać normalizacji z użyciem 'Scalera' wyuczonego na zbiorze treningowym

iris_X_test_scaled = scaler.transform(iris_X_test)

In [ ]:
yhat_iris_test = model.predict(iris_X_test_scaled)

In [ ]:
np.mean(tf.metrics.categorical_accuracy(iris_Y_test, yhat_iris_test))

In [ ]:
'''Przykład 2 - Prognozowanie wartości numerycznych (cen mieszkań)

Klasyfikacja jest przedstawiana jako klasyczny przykład zastosowania prostych sieci neuronowych.
Innym jest regresja, czyli prognozowanie wartości numerycznych. Posłużymy się zbiorem danych 'Boston housing',
gdzie na podstawiec ceh mieszkania będziemy prognozować jego cenę.

1. Za pomocą pakietu scikit-learn załadujemy dane ze zbioru Boston housing.
2. Przekonwertujemy je do postaci Data Frame, żeby móc wygodnie operować na danych.
3. Przprowadzimy podział na zbiór treningowy i testowy.
4. Przprowadzimy normalizację danych.'''

In [ ]:
# wczytanie i wstępna obróbka
boston = load_boston()
boston_df = pd.DataFrame(boston.data, columns=boston.feature_names)
boston_y = pd.Series(boston.target)

In [ ]:
boston_df.head(3)

In [ ]:
# podział na zbiór testowy i uczący

boston_X_train, boston_X_test, boston_y_train, boston_y_test = train_test_split(boston_df, boston_y, train_size=0.8, random_state=123)

In [ ]:
boston_X_train.shape, boston_y_train.shape

In [ ]:
# normalizacja danych
scaler_boston = StandardScaler()
boston_X_trains = scaler_boston.fit_transform(boston_X_train)
boston_X_trains = pd.DataFrame(boston_X_trains, columns=boston_X_train.columns)

In [ ]:
''' Wykorzystanie biblioteki do zbudowania sieci

Kroki budowy sieci z użyciem biblioteki są analogiczne jak w przypadku klasyfikacji. Jest kilka różnic:
1. W tym przypadku, jako aktywacji ostatniej warstwy użyjemu ReLu, ponieważ prognozujemy wartości numeryczne.
2. Funkcją kosztu będzie Mean Squared Error (MSE), ponieważ mamy do czynienia z regresją.
3. Dodamy (jako podgląd) średni bezwzględny błąd procentowy (MAPE), żeby wiedzieć o ile % ceny docelowej myli się nasz model.'''

In [ ]:
nx_boston = boston_X_train.shape[1] # liczba cech
nh_boston = 12 # liczba neuronów w warstwie ukrytej

model_boston = tf.keras.models.Sequential()
model_boston.add(tf.keras.layers.Dense(input_dim=nx_boston, units=nh_boston, activation='relu'))
model_boston.add(tf.keras.layers.Dense(units=1, activation='relu'))
model_boston.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.9), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=[tf.keras.losses.MeanSquaredError(name="MSE"), tf.keras.losses.MeanAbsolutePercentageError(name="MAPE")])

print(model_boston.summary())
tf.keras.utils.plot_model(model_boston, show_shapes=True, show_layer_names=True, rankdir='LR')

In [ ]:
history_boston = model_boston.fit(boston_X_trains, boston_y_train, epochs=20, batch_size=64)

In [ ]:
plt.plot(history_boston.history["loss"])
plt.title("Błąd średniokwadratowy cen")
plt.xlabel("Iteracja")
plt.ylabel("MSE")

In [ ]:
plt.plot(history_boston.history["MSE"])
plt.title("Błąd [%] cen")
plt.xlabel("Iteracja")
plt.ylabel("MAPE")

In [ ]:
'''Predykcja na zbiorze testowym

Podobnie jako to miało miejsce, w przypadku klasyfikacji kwiatów irysa - dokonamy predykcji na zbiorze testowym.
Najpierw należy znormalizować dane z wykorzystaniem 'Scalera' nauczonego na danych treningowych'''

boston_X_test_scaled = scaler_boston.transform(boston_X_test)

In [ ]:
yhat_boston = model_boston.predict(boston_X_test_scaled).squeeze()

In [ ]:
mean_squared_error(boston_y_test, yhat_boston)

In [ ]:
tf.keras.metrics.mean_absolute_percentage_error(boston_y_test, yhat_boston).numpy()